In [1]:
import os,re,json
import polars as pl
import plotly.express as px

os.getcwd()
# os.chdir('./')

'/home/kickd/Documents/to_ceres/Virus-project-/Virus_project_files'

In [2]:
# really quickly aggregate the model summaries for plotting and such
models = os.listdir('./models/prod')
models

['rf-Vitis_vinifera-kmer3-bpgenome-fold2__metrics.json',
 'brf-Zea_mays-kmer3-bpgenome-fold0.pkl',
 'GNBC-Zea_mays-kmer3-bpgenome-fold2__metrics.json',
 'bknn-Spodoptera_frugiperda-kmer3-bpgenome-fold0__tst.png',
 'rf-Drosophila_melanogaster-kmer3-bpgenome-fold2__metrics.json',
 'hgb-Glycine_max-kmer3-bpgenome-fold1__metrics.json',
 'brf-Drosophila_melanogaster-kmer3-bpgenome-fold2__trn.png',
 'hgb-Vitis_vinifera-kmer3-bpgenome-fold4__metrics.json',
 'svmr-Zea_mays-kmer3-bpgenome-fold0__metrics.json',
 'hgb-Vitis_vinifera-kmer3-bpgenome-fold0__metrics.json',
 'rf-Glycine_max-kmer3-bpgenome-fold1__trn.png',
 'brf-Zea_mays-kmer3-bpgenome-fold1.pkl',
 'brf-Spodoptera_frugiperda-kmer3-bpgenome-fold3.pkl',
 'GNBC-Spodoptera_frugiperda-kmer3-bpgenome-fold0__trn.png',
 'GNBC-Glycine_max-kmer3-bpgenome-fold2__trn.png',
 'rf-Vitis_vinifera-kmer3-bpgenome-fold2__tst.png',
 'GNBC-Vitis_vinifera-kmer3-bpgenome-fold2__predictions.parquet',
 'GNBC-Glycine_max-kmer3-bpgenome-fold1__trn.png',
 'bknn-G

In [3]:
metrics = [e for e in models if re.match(r'.+\.json', e)]

out = []
for j in metrics:
    with open(f'./models/prod/{j}', 'r') as f:
        out.append(json.load(f))

In [4]:
tmp = {
     'file':[],
     'model':[],
     'target':[],
     'kmer':[],
     'contig':[],
     'fold':[],
    }

for e in metrics:
    _ = e.split('__')[0].split('-')
    tmp['file'].append(e)
    tmp['model'].append(_[0])
    tmp['target'].append(_[1])
    tmp['kmer'].append(_[2])
    tmp['contig'].append(_[3])
    tmp['fold'].append(_[4])


metrics = pl.concat([
    pl.DataFrame(tmp),
    pl.concat([pl.DataFrame(e) for e in out])
    ], 
    how='horizontal')

In [5]:
metrics.write_parquet('./models/prod/all__metrics.parquet')

In [6]:
predictions = [e for e in models if re.match(r'.+fold\d+__.+\.parquet', e)]

p = predictions[0]

In [7]:
def _parse_filename(fn):
    _ = fn.split('__')[0].split('-')
    tmp = {'file': [fn],
    'model': [_[0]],
    'target': [_[1]],
    'kmer': [_[2]],
    'contig': [_[3]],
    'fold': [_[4]]}
    return(tmp)



In [8]:
# parse file name as df, join with data save as list and concat
_ = [(pl.DataFrame(_parse_filename(fn=p)).with_columns(join_on_this = pl.lit(True))
 .join(pl.read_parquet(f"./models/prod/{p}").with_columns(join_on_this = pl.lit(True)), 
       on='join_on_this').drop(pl.col('join_on_this'))
       ) for p in predictions]

In [9]:
predictions = pl.concat(_)

In [10]:
predictions.write_parquet('./models/prod/all__predictions.parquet')

In [11]:
predictions.shape

(16435250, 11)

In [12]:
_ = (
    predictions
    .filter(pl.col('TestSet') == True)
    .drop('file', 'TestSet')
    .group_by([pl.col(e) for e in ['model', 'target', 'kmer', 'contig', 'Header', 'Label']])
    .agg(
        pl.col('Yhat').sum(), 
        pl.col('ProbPos').mean(), 
        )
    )

In [13]:
_ = _.pivot('target', index = ['model', 'kmer', 'contig', 'Header', 'Label'], values= ['Yhat', 'ProbPos'])
_


model,kmer,contig,Header,Label,Yhat_Spodoptera_frugiperda,Yhat_Vitis_vinifera,Yhat_Glycine_max,Yhat_Zea_mays,Yhat_Drosophila_melanogaster,ProbPos_Spodoptera_frugiperda,ProbPos_Vitis_vinifera,ProbPos_Glycine_max,ProbPos_Zea_mays,ProbPos_Drosophila_melanogaster
str,str,str,str,str,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
"""rf""","""kmer3""","""bpgenome""","""FJ943293.1 |Grapevine rupestri…","""Vitis_vinifera""",0,1,0,0,0,0.000013,0.98996,0.0,0.01,0.00008
"""rf""","""kmer3""","""bpgenome""","""KF744395.1 |Maize chlorotic mo…","""Zea_mays""",0,0,0,13,0,0.000071,0.001538,0.000769,0.996921,0.000471
"""bknn""","""kmer3""","""bpgenome""","""MT454064.1 |Grapevine rupestri…","""Vitis_vinifera""",0,28,1,0,0,0.0,0.992857,0.057143,0.0,0.0
"""hgb""","""kmer3""","""bpgenome""","""KX034939.1 |Grapevine fanleaf …","""Vitis_vinifera""",0,11,0,0,0,0.000002,0.9984,0.001663,0.009751,0.000648
"""brf""","""kmer3""","""bpgenome""","""MZ572797.1 |Grapevine leafroll…","""Vitis_vinifera""",0,1,0,0,0,0.008275,0.946353,0.061379,0.051435,0.031355
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""brf""","""kmer3""","""bpgenome""","""KP401799.1 |Grapevine leafroll…","""Vitis_vinifera""",0,1,0,0,0,0.005207,0.920698,0.018443,0.076839,0.070568
"""hgb""","""kmer3""","""bpgenome""","""HQ655071.1 |Drosophila melanog…","""Drosophila_melanogaster""",0,0,0,0,18,0.000032,0.003153,0.001727,0.006619,0.995474
"""brf""","""kmer3""","""bpgenome""","""KF013803.1 |Grapevine virus A …","""Vitis_vinifera""",0,1,0,0,0,0.004963,0.92179,0.03043,0.077788,0.08151


In [14]:
_.write_parquet('./models/prod/all_agg_pivot__predictions.parquet')

In [15]:
# what are the best models for each species?

# use metrics to get the best models to examine

_ = (metrics
.select(  ['model', 'target', 'kmer', 'contig', 'fold', 'accuracy_tst'])
.group_by(['model', 'target', 'kmer', 'contig'])
.agg(pl.col('accuracy_tst').mean())
)

best_models = (_
               .group_by(['model', 'target'])
               .agg(pl.col('accuracy_tst').max())
               ).join(_, how='inner', on=['model',  'target', 'accuracy_tst'])

best_models = best_models.select(['model',  'target', 'kmer', 'contig', 'accuracy_tst'])
best_models

model,target,kmer,contig,accuracy_tst
str,str,str,str,f64
"""hgb""","""Glycine_max""","""kmer3""","""bpgenome""",0.985302
"""svmr""","""Spodoptera_frugiperda""","""kmer3""","""bpgenome""",0.997992
"""bknn""","""Vitis_vinifera""","""kmer3""","""bpgenome""",0.990389
"""svmr""","""Drosophila_melanogaster""","""kmer3""","""bpgenome""",0.99201
"""svml""","""Glycine_max""","""kmer3""","""bpgenome""",0.892987
…,…,…,…,…
"""GNBC""","""Zea_mays""","""kmer3""","""bpgenome""",0.817159
"""svml""","""Drosophila_melanogaster""","""kmer3""","""bpgenome""",0.920515
"""svmr""","""Zea_mays""","""kmer3""","""bpgenome""",0.989288


In [16]:
best_models.write_csv('./models/prod/all_best_models_summary.csv')

In [17]:
# get all the predictions from only the best models 
_ = (
    best_models
    .drop('accuracy_tst')
    .join(
        (predictions
         .filter(pl.col('TestSet') == True)
         .drop('file', 'TestSet')
        ), how= 'left', on = ['model',  'target', 'kmer', 'contig'])
)
_

model,target,kmer,contig,fold,Header,Label,Yhat,ProbPos
str,str,str,str,str,str,str,i16,f64
"""hgb""","""Glycine_max""","""kmer3""","""bpgenome""","""fold0""","""OR820571.1 |Chaq virus isolate…","""Drosophila_melanogaster""",0,0.029135
"""hgb""","""Glycine_max""","""kmer3""","""bpgenome""","""fold0""","""OR820571.1 |Chaq virus isolate…","""Drosophila_melanogaster""",0,0.006816
"""hgb""","""Glycine_max""","""kmer3""","""bpgenome""","""fold0""","""OR820571.1 |Chaq virus isolate…","""Drosophila_melanogaster""",0,0.022596
"""hgb""","""Glycine_max""","""kmer3""","""bpgenome""","""fold0""","""OR820572.1 |Dansoman virus iso…","""Drosophila_melanogaster""",0,0.00002
"""hgb""","""Glycine_max""","""kmer3""","""bpgenome""","""fold0""","""OR820572.1 |Dansoman virus iso…","""Drosophila_melanogaster""",0,0.0005
…,…,…,…,…,…,…,…,…
"""rf""","""Zea_mays""","""kmer3""","""bpgenome""","""fold0""","""AJ310114.1 |Sugarcane mosaic v…","""Zea_mays""",1,0.97
"""rf""","""Zea_mays""","""kmer3""","""bpgenome""","""fold0""","""AJ310114.1 |Sugarcane mosaic v…","""Zea_mays""",1,0.979982
"""rf""","""Zea_mays""","""kmer3""","""bpgenome""","""fold0""","""AF395135.1 |Maize dwarf mosaic…","""Zea_mays""",1,0.61


In [18]:
# we only need the target to identify the parameters so we can select very few cols
_ = _.select('model', 'target', 'Header', 'Label', 'Yhat', 'ProbPos').group_by(['model', 'target', 'Header', 'Label']).agg(
    pl.col('Yhat').mean(), 
    pl.col('ProbPos').mean(), 
)

_ 
# ).pivot('target', index = ['model', 'target', 'Header', 'Label'], values= ['Yhat', 'ProbPos'])


model,target,Header,Label,Yhat,ProbPos
str,str,str,str,f64,f64
"""GNBC""","""Spodoptera_frugiperda""","""FM161881.1 |Mungbean yellow mo…","""Glycine_max""",0.285714,0.228263
"""brf""","""Zea_mays""","""KF297335.1 |Soybean mosaic vir…","""Glycine_max""",0.0,0.135221
"""brf""","""Vitis_vinifera""","""MG731647.1 |Grapevine fanleaf …","""Vitis_vinifera""",1.0,0.880826
"""bknn""","""Spodoptera_frugiperda""","""KC861206.1 |Grapevine virus B …","""Vitis_vinifera""",0.0,0.0
"""rf""","""Drosophila_melanogaster""","""OM286794.1 |Grapevine leafroll…","""Vitis_vinifera""",0.0,0.05036
…,…,…,…,…,…
"""rf""","""Zea_mays""","""MH999409.1 |Rice black streake…","""Zea_mays""",1.0,0.974809
"""rf""","""Zea_mays""","""MH521112.2 |Grapevine leafroll…","""Vitis_vinifera""",0.0,0.002814
"""bknn""","""Spodoptera_frugiperda""","""KY381153.1 |UNVERIFIED: Grapev…","""Vitis_vinifera""",0.0,0.0


In [19]:
# based on the shape fo the dfs there are ever so slightly more than 13 observations being collapsed. This could be from slight imballences in cvs.
# _.select('Yhat').unique()

442994/33890

13.07152552375332

In [20]:
# per target model == label

# example for one model 
(
    _
    .filter(pl.col('Yhat') > 0.5)
    .filter(pl.col('target') == 'Vitis_vinifera')
    .group_by('Label')
    .count()
)

# .group_by('Label', )

/tmp/ipykernel_29502/3787948167.py:9: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  .count()


Label,count
str,u32
"""Vitis_vinifera""",18868
"""Glycine_max""",695
"""Zea_mays""",637
"""Drosophila_melanogaster""",232


In [21]:
best_models_pospred = (
    _
    .filter(pl.col('Yhat') > 0.5)
    .group_by('target', 'Label')
    .len()
    .sort('target', 'Label')
)
best_models_pospred

target,Label,len
str,str,u32
"""Drosophila_melanogaster""","""Drosophila_melanogaster""",2197
"""Drosophila_melanogaster""","""Glycine_max""",40
"""Drosophila_melanogaster""","""Spodoptera_frugiperda""",49
"""Drosophila_melanogaster""","""Vitis_vinifera""",118
"""Drosophila_melanogaster""","""Zea_mays""",551
…,…,…
"""Zea_mays""","""Drosophila_melanogaster""",347
"""Zea_mays""","""Glycine_max""",182
"""Zea_mays""","""Spodoptera_frugiperda""",27


In [22]:
best_models_pospred.write_csv('./models/prod/all_best_models_pospred_summary.csv')

In [23]:
# What are the ones we're suprised by?
# these are the ones that should be followed up.
best_models_falsepos = (
    _
    .filter(pl.col('Yhat') > 0.5)
    .filter(pl.col('target') != pl.col('Label'))
    .sort('target', 'Label')
)

best_models_falsepos.write_csv('./models/prod/all_best_models_falsepos.csv')

In [24]:
from   ax.service.ax_client import AxClient

In [25]:
# really quickly aggregate the model summaries for plotting and such
axs = os.listdir('./models/tune')


In [26]:
def _parse_filename(fn):
    _ = fn.split('__')[0].split('-')
    tmp = {'file': [fn],
    'model': [_[0]],
    'target': [_[1]],
    'kmer': [_[2]],
    'contig': [_[3]],
    'fold': [_[4]]}
    return(tmp)



In [27]:
_parse_filename(fn = j)

{'file': ['svmr-Glycine_max-kmer3-bpgenome-fold4__metrics.json'],
 'model': ['svmr'],
 'target': ['Glycine_max'],
 'kmer': ['kmer3'],
 'contig': ['bpgenome'],
 'fold': ['fold4']}

In [28]:
j = axs[0]

_ = []
for j in axs:

    ax_client = (AxClient.load_from_json_file(filepath = f"./models/tune/{j}"))

    hyps = pl.DataFrame(ax_client.get_trials_data_frame())

    hyps = pl.DataFrame(_parse_filename(fn=j)).with_columns(join_on_this = pl.lit(True)).join(
                                hyps.with_columns(join_on_this = pl.lit(True)),
                                on='join_on_this').drop(pl.col('join_on_this'))
    _.append(hyps)

[INFO 01-30 12:27:58] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[WARNING 01-30 12:27:58] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.
[INFO 01-30 12:27:58] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[WARNING 01-30 12:27:58] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.
[INFO 01-30 12:27:58] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[WARNING 01-30 12:27:58] ax.service.utils.report_utils: Column reason missing for all trials. Not a

In [29]:
# when we were only looking at one model we could do this:
# _ = pl.concat(_)
# Now we have to instead 
# 1. Find all of the models in each dataset and then concat those. 
_model_type = [pl.Series(e.select(pl.first('model'))).to_list()[0] for e in _]
_ = {key:pl.concat([_[i] for i in range(len(_model_type)) if _model_type[i] == key]) for key in set(_model_type)}

# 2. After that's done then full join with nans for non-relevalnt hyperparams
out = None
for key in _.keys():
    if type(out) is type(None):
        out = _[key]
    else:
        out = (
            out.join(
                _[key], 
                how='full', 
                # use the intersection between columns to determine the fields to join on
                on= list(set(out.columns).intersection(set(_[key].columns))), 
                coalesce=True
                )
                )
_ = out

In [30]:
_.write_parquet('./models/prod/all__hyperparameters.parquet')